In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import subprocess
from functools import reduce

In [2]:
from ufs2arco import sources

In [4]:
gfs = sources.GFSArchive(
    t0={"start": "2015-12-31T00", "end": "2024-12-31T00", "freq": "1YE"},
    fhr={"start": 0, "end": 6, "step": 6},
)

In [5]:
def pull_both_local(dims, cache_dir):
    a = gfs._open_local(
        dims=dims,
        file_suffix="",
        cache_dir="./gribcache",
    )
    b = gfs._open_local(
        dims=dims,
        file_suffix="b",
        cache_dir="./gribcache",
    )
    return a,b 

In [7]:
def open_datasets(dims, cache_dir, **kwargs):

    dsdict = {}
    for file_suffix in ["", "b"]:
        try:
            dsdict[file_suffix] = gfs.open_grib(
                dims=dims,
                file_suffix=file_suffix,
                cache_dir="./gribcache",
                **kwargs,
            ) 
        except:
            dsdict[file_suffix] = None

    if all([xx is not None for xx in dsdict.values()]):
        alist = set(sorted(dsdict[""].data_vars))
        blist = set(sorted(dsdict["b"].data_vars))
        xds = xr.merge(list(dsdict.values()))
        
    elif dsdict[""] is not None or dsdict["b"] is not None:
        if dsdict[""] is not None:
            alist = set(sorted(dsdict[""].data_vars))
            blist = set()
            xds = dsdict[""]
        else:
            alist = set()
            blist = set(sorted(dsdict["b"].data_vars))
            xds = dsdict["b"]
    else:
        raise

    varlist = set(sorted(xds.data_vars))
    onlyA = alist - varlist.intersection(blist)
    onlyB = blist - varlist.intersection(alist)
    for key in varlist:
        if key in onlyA:
            xds[key].attrs["file_suffix"] = [""]
        elif key in onlyB:
            xds[key].attrs["file_suffix"] = ["b"]
        else:
            xds[key].attrs["file_suffix"] = ["", "b"]
    return xds

### First, figure out surface stepTypes available

In [9]:
dsdict = {}
for t0 in gfs.t0:
    dsdict[t0] = {}

    for fhr in gfs.fhr:
        step_types = []
        for file_suffix in ["", "b"]:
            print(f"Reading (t0, fhr) = ({str(t0)}, {int(fhr)})")
            a = gfs._open_local(
                dims={"t0": t0, "fhr": fhr},
                file_suffix=file_suffix,
                cache_dir="./gribcache",
            )
            output = subprocess.check_output(
                ["grib_ls", "-p", "typeOfLevel,stepType", a],
                stderr=subprocess.DEVNULL
            ).decode()
    
            for line in output.splitlines():
                parts = line.strip().split()
                if len(parts) >= 2:
                    type_of_level, step_type = parts[-2], parts[-1]
                    if type_of_level == typeOfLevel:
                        step_types.append(step_type)
        dsdict[t0][fhr] = sorted(set(step_types))

Reading (t0, fhr) = (2015-12-31 00:00:00, 0)
Reading (t0, fhr) = (2015-12-31 00:00:00, 0)
Reading (t0, fhr) = (2015-12-31 00:00:00, 6)
Reading (t0, fhr) = (2015-12-31 00:00:00, 6)
Reading (t0, fhr) = (2016-12-31 00:00:00, 0)
Reading (t0, fhr) = (2016-12-31 00:00:00, 0)
Reading (t0, fhr) = (2016-12-31 00:00:00, 6)
Reading (t0, fhr) = (2016-12-31 00:00:00, 6)
Reading (t0, fhr) = (2017-12-31 00:00:00, 0)
Reading (t0, fhr) = (2017-12-31 00:00:00, 0)
Reading (t0, fhr) = (2017-12-31 00:00:00, 6)
Reading (t0, fhr) = (2017-12-31 00:00:00, 6)
Reading (t0, fhr) = (2018-12-31 00:00:00, 0)
Reading (t0, fhr) = (2018-12-31 00:00:00, 0)
Reading (t0, fhr) = (2018-12-31 00:00:00, 6)
Reading (t0, fhr) = (2018-12-31 00:00:00, 6)
Reading (t0, fhr) = (2019-12-31 00:00:00, 0)
Reading (t0, fhr) = (2019-12-31 00:00:00, 0)
Reading (t0, fhr) = (2019-12-31 00:00:00, 6)
Reading (t0, fhr) = (2019-12-31 00:00:00, 6)
Reading (t0, fhr) = (2020-12-31 00:00:00, 0)
Reading (t0, fhr) = (2020-12-31 00:00:00, 0)
Reading (t

In [10]:
for t0, fdict in dsdict.items():
    print(f"t0 = {t0}")
    print(f"\t{fdict[0]} \t {fdict[6]}")

t0 = 2015-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2016-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2017-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2018-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2019-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2020-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2021-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2022-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2023-12-31 00:00:00
	['instant'] 	 ['instant']
t0 = 2024-12-31 00:00:00
	['instant'] 	 ['instant']


Just instant

### Now, get the variables

In [14]:
vdict = {}
for stepType in ["instant"]:
    vdict[stepType] = {}
    for t0 in gfs.t0:
        vdict[stepType][t0] = {}
        dslist = []
        varlist = []
        for fhr in gfs.fhr:
            if fhr == 0 and stepType != "instant":
                vdict[stepType][t0][fhr] = set(xds.data_vars)
            else:
                xds = open_datasets(
                    dims={"t0": t0, "fhr": fhr},
                    cache_dir="./gribcache",
                    filter_by_keys={
                        "typeOfLevel": typeOfLevel,
                        "stepType": stepType,
                    },
                )
                vdict[stepType][t0][fhr] = set(xds.data_vars)

Ignoring index file './gribcache/549a6ef68b157888995accc532cca9747636064a49303c107f4368beb813750a.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/da481ccec513cce761363ca7fa8f343791a3d47a25654021bc462f8fd2ff7100.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/0b9c83dcf8848d5b45d0b5214a694e00f08eb70e14eeca7cb2d519963fee684c.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/7add9606d27fde3bc48fb1234843633bb2af175f6cc11d71dc11c12fed8a1302.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/ac3e1a35d87d1750ba226768c94b8717cabca983eb79bc4a98c4381fc0885d6f.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/1e12f111184c8a5847c61734d98e1ec296afbc9486c1f513b9fa8a507871ce5f.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/b7ac984740ae4178927df957c26fcac69f83afd9c3878bef0694af7ed00f64e5.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/2504c18e7f00d3436bf32dcb4cbde67e0ba679719ef010b25bf00cdd0

In [15]:
vdict["instant"]

{Timestamp('2015-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2016-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2017-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2018-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2019-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2020-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2021-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2022-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2023-12-31 00:00:00'): {np.int64(0): {'mslet', 'prmsl'},
  np.int64(6): {'mslet', 'prmsl'}},
 Timestamp('2024-12-31 00:00:00'): {np.int64(0): {'msle

In [16]:
for stepType, d2 in vdict.items():
    for t0, d3 in d2.items():
        intersect = reduce(set.intersection, [set(x) for x in d3.values()]) 
        if len(d3[0] - intersect) > 0:
            print(f"More in analysis t0 = {t0}, stepType = {stepType}")
        if len(d3[6] - intersect) > 0:
            print(f"More in forecast t0 = {t0}, stepType = {stepType}")

These should all be the same...

### Get the common variables in each

In [17]:
intersect_analysis = {
    key: sorted(reduce(set.intersection, [set(x[0]) for x in vdict[key].values()]))
    for key in vdict.keys()
}

In [18]:
intersect_analysis

{'instant': ['mslet', 'prmsl']}

In [19]:
intersect_forecast = {
    key: sorted(reduce(set.intersection, [set(x[6]) for x in vdict[key].values()]))
    for key in vdict.keys()
}

In [20]:
intersect_forecast

{'instant': ['mslet', 'prmsl']}

In [21]:
for key in intersect_forecast.keys():
    print(f" --- {key} --- ")
    print("analysis - forecast: ", set(intersect_analysis[key]) - set(intersect_forecast[key]))
    print("forecast - analysis: ",set(intersect_forecast[key]) - set(intersect_analysis[key]))
    print()

 --- instant --- 
analysis - forecast:  set()
forecast - analysis:  set()



In [22]:
intersect = {
    0: intersect_analysis,
    6: intersect_forecast,
}

all good

### Get the unique per t0 variables

In [23]:
for stepType, d2 in vdict.items():
    print(f"stepType = {stepType}")
    for t0, d3 in d2.items():
        for fhr, d4 in d3.items():
            unique = set(d4) - set(intersect[fhr][stepType])
            if len(unique) > 0:
                print(f"\t{t0}, fhr = {fhr}")
                print(f"\t\t{unique}")

stepType = instant


### Now, let's open a dataset, get these variables, and write out an updated dict

### First, instant stepType

Note that most of the time `cnwat` and `sithick` are only in `"b"`, but sometimes they are in both...
When they're in both, they appear to be exactly the same, since the dataset merges with no problem.

So... seems safe to set the file suffix for these to `"b"`

In [35]:
stepType = "instant"
itime = 0
dsdict2 = {}
ads = open_datasets(
    dims={"t0": gfs.t0[itime], "fhr": gfs.fhr[0]},
    cache_dir="./gribcache",
    filter_by_keys={
        "typeOfLevel": typeOfLevel,
        "stepType": stepType,
    },
)
fds = open_datasets(
    dims={"t0": gfs.t0[itime], "fhr": gfs.fhr[1]},
    cache_dir="./gribcache",
    filter_by_keys={
        "typeOfLevel": typeOfLevel,
        "stepType": stepType,
    },
)

ads = ads[sorted(intersect[0][stepType])]
fds = fds[sorted(intersect[6][stepType])]
xds = xr.concat([ads, fds], dim="fhr")

if "unknown" in xds:
    xds = xds.drop_vars("unknown")

dsdict2["instant"] = xds.copy()

Ignoring index file './gribcache/549a6ef68b157888995accc532cca9747636064a49303c107f4368beb813750a.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/da481ccec513cce761363ca7fa8f343791a3d47a25654021bc462f8fd2ff7100.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/0b9c83dcf8848d5b45d0b5214a694e00f08eb70e14eeca7cb2d519963fee684c.5b7b6.idx' older than GRIB file
Ignoring index file './gribcache/7add9606d27fde3bc48fb1234843633bb2af175f6cc11d71dc11c12fed8a1302.5b7b6.idx' older than GRIB file


In [36]:
newdict = {}
for stepType, xds in dsdict2.items():
    for varname in sorted(xds.data_vars):
        newdict[varname] = {
            "filter_by_keys": {
                "typeOfLevel": xds[varname].GRIB_typeOfLevel,
                "paramId": xds[varname].GRIB_paramId,
            },
            "long_name": xds[varname].long_name,
            "file_suffixes": xds[varname].file_suffix,
            "forecast_only": stepType != "instant",
        }
        if xds[varname].GRIB_typeOfLevel == "heightAboveGround":
            newdict[varname]["filter_by_keys"]["level"] = xds[varname].attrs["GRIB_level"]
        elif xds[varname].GRIB_typeOfLevel == "surface":
            newdict[varname]["filter_by_keys"]["stepType"] = xds[varname].attrs["GRIB_stepType"]
        if "original_name" in xds[varname].attrs:
            newdict[varname]["original_name"] = xds[varname].original_name

In [37]:
newdict = {key: newdict[key] for key in sorted(list(newdict.keys()))}

In [38]:
newdict

{'mslet': {'filter_by_keys': {'typeOfLevel': 'meanSea', 'paramId': 260317},
  'long_name': 'MSLP (Eta model reduction)',
  'file_suffixes': [''],
  'forecast_only': False},
 'prmsl': {'filter_by_keys': {'typeOfLevel': 'meanSea', 'paramId': 260074},
  'long_name': 'Pressure reduced to MSL',
  'file_suffixes': [''],
  'forecast_only': False}}

In [39]:
import yaml

In [40]:
sources.__path__[0]

'/Users/tsmith/work/ufs2arco/ufs2arco/sources'

In [41]:
with open(f"{sources.__path__[0]}/reference.gfs.yaml", "r") as f:
    reference = yaml.safe_load(f)

In [42]:
updated = reference.copy()

In [43]:
updated.update(newdict)


In [44]:
updated = {key: updated[key] for key in sorted(updated.keys())}

In [45]:
with open("reference.gfs.yaml", "w") as f:
    yaml.dump(updated, f)